# Fetch Data

This project relies on [explain here: data sources it uses]

In [1]:
from scripts.data_fetchers.fetch_firm_ueis import read_sam_json, fetch_sam_data

2024-12-14 16:45:35.720 | INFO     | scripts.config:<module>:11 - PROJ_ROOT path is: /Users/jaspercooper/Dropbox/01_Projects/01_python_projects/predict_marketing_success


In [2]:
fetch_sam_data()

{'totalRecords': 5874, 'entityData': [{'entityRegistration': {'samRegistered': 'Yes', 'ueiSAM': 'TKB9GELBZGG4', 'entityEFTIndicator': None, 'cageCode': '9YNT8', 'dodaac': None, 'legalBusinessName': 'CASBRO DEVELOPMENT, LLC', 'dbaName': None, 'purposeOfRegistrationCode': 'Z2', 'purposeOfRegistrationDesc': 'All Awards', 'registrationStatus': 'Active', 'registrationDate': '2024-07-17', 'lastUpdateDate': '2024-09-30', 'registrationExpirationDate': '2025-09-26', 'activationDate': '2024-09-30', 'ueiStatus': 'Active', 'ueiExpirationDate': None, 'ueiCreationDate': '2024-07-17', 'noPublicDisplayFlag': 'Y', 'exclusionStatusFlag': None, 'exclusionURL': None, 'dnbOpenData': None}, 'coreData': {'entityInformation': {'entityURL': 'https://www.casbrodc.com', 'entityDivisionName': None, 'entityDivisionNumber': None, 'entityStartDate': '2014-05-07', 'fiscalYearEndCloseDate': '12/31', 'submissionDate': '2024-09-26'}, 'physicalAddress': {'addressLine1': '2858 HIDEAWAY RD', 'addressLine2': None, 'city': '

In [9]:
data = read_sam_json()

{'totalRecords': 5875, 'entityData': [{'entityRegistration': {'samRegistered': 'Yes', 'ueiSAM': 'TKB9GELBZGG4', 'entityEFTIndicator': None, 'cageCode': '9YNT8', 'dodaac': None, 'legalBusinessName': 'CASBRO DEVELOPMENT, LLC', 'dbaName': None, 'purposeOfRegistrationCode': 'Z2', 'purposeOfRegistrationDesc': 'All Awards', 'registrationStatus': 'Active', 'registrationDate': '2024-07-17', 'lastUpdateDate': '2024-09-30', 'registrationExpirationDate': '2025-09-26', 'activationDate': '2024-09-30', 'ueiStatus': 'Active', 'ueiExpirationDate': None, 'ueiCreationDate': '2024-07-17', 'noPublicDisplayFlag': 'Y', 'exclusionStatusFlag': None, 'exclusionURL': None, 'dnbOpenData': None}, 'coreData': {'entityInformation': {'entityURL': 'https://www.casbrodc.com', 'entityDivisionName': None, 'entityDivisionNumber': None, 'entityStartDate': '2014-05-07', 'fiscalYearEndCloseDate': '12/31', 'submissionDate': '2024-09-26'}, 'physicalAddress': {'addressLine1': '2858 HIDEAWAY RD', 'addressLine2': None, 'city': '